In [1]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q datasets
!pip install -q gdown
!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle kernels output krittiwitkampradam/xglm-multi-gpus-pure-torch -p /path

Saving kaggle.json to kaggle.json
Output file downloaded to /path/checkpoint/config.json
Output file downloaded to /path/checkpoint/generation_config.json
Output file downloaded to /path/checkpoint/pytorch_model.bin
Output file downloaded to /path/output.jsonl
Kernel log downloaded to /path/xglm-multi-gpus-pure-torch.log 


In [3]:
from transformers import XGLMTokenizer, XGLMForCausalLM, GenerationConfig
import torch
def load_model(base_model:str="ohmreborn/finetune-xglm-564M"):
    tokenizer = XGLMTokenizer.from_pretrained(base_model)
    model = XGLMForCausalLM.from_pretrained('/path/checkpoint/',torch_dtype=torch.float16)
    return model,tokenizer
model,tokenizer = load_model()

In [4]:
model

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256010, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [38]:
import torch
generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
    )
device = torch.device('cuda')
model.to(device)

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256010, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [39]:
import re
def gen(prompt,generation_config=generation_config):
  prompt = f'{prompt}<human>:'
  inputs = tokenizer(prompt, return_tensors="pt")
  input_ids = inputs["input_ids"]
  input_ids = input_ids.to(device)
  def get_res(inputs):
    with torch.no_grad():
      generation_output = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
          max_new_tokens=512,
      )
    output_1_ids = generation_output.sequences[0]
    output = tokenizer.decode(output_1_ids)  
    return output
  output = get_res(inputs)
  if '<bot>:' in output:
    res = re.split('<human>:|<bot>:|</s>',output)
    return f'<human>: {res[2]} \n<bot>: {res[3]}'
  else:
    bot_prompt = re.split(tokenizer.eos_token,output)[1]
    bot_prompt = f'{bot_prompt} <bot>:'

    inputs = tokenizer(bot_prompt, return_tensors="pt")
    input_ids = inputs["input_ids"]
    input_ids = input_ids.to(device)
    output = get_res(inputs)
    res = re.split('<human>:|<bot>:|</s>',output)
    return f'<human>: {res[2]} \n<bot>: {res[2]}'

In [40]:
import gdown

url = "https://drive.google.com/uc?export=download&id=1tIsYn3Mb7rLb8OfXxEv0_cwwEr3AjZUl"
output = 'output.jsonl'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?export=download&id=1tIsYn3Mb7rLb8OfXxEv0_cwwEr3AjZUl
To: /content/output.jsonl
100%|██████████| 283M/283M [00:04<00:00, 62.5MB/s]


'output.jsonl'

In [14]:
from datasets import load_dataset
datasets = load_dataset('json',data_files='output.jsonl')

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
import multiprocessing
cpu_cores = multiprocessing.cpu_count()

In [42]:
def format_prompt(prompt):
    return {'prompt':f"{prompt['Background:']}{prompt['<human>:_<bot>:']}"}
# format data like  context ......
def preprocess(prompt):
    data = tokenizer(
        prompt['prompt'],
        truncation=True,
        max_length=128,
        padding=False,
        return_tensors=None,
    )
    data['input_ids'].append(tokenizer.eos_token_id)
    data['attention_mask'].append(1)
    data['labels'] = data['input_ids'].copy()
    return data

In [17]:
from datasets import Dataset
datasets = Dataset.from_dict(datasets['train'][:1_000]) # sample data for test
# datasets = datasets['train']
datasets = datasets.map(format_prompt,num_proc=cpu_cores,batched=False)
datasets = datasets.map(lambda x:{'token':len(tokenizer.tokenize(x['prompt']))},num_proc=cpu_cores,batched=False)
datasets = datasets.filter(lambda x:x['token']<512-1,num_proc=cpu_cores)
# datasets = datasets.map(preprocess,remove_columns=['prompt','token','Background:', '<human>:_<bot>:'],num_proc=cpu_cores) 

datasets

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['Background:', '<human>:_<bot>:', 'prompt', 'token'],
    num_rows: 525
})

In [29]:
print(datasets[0]['prompt'])

 Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins, or one of the member proteins of that superfamily. α- and β-tubulins polymerize into microtubules, a major component of the eukaryotic cytoskeleton. Microtubules function in many essential cellular processes, including mitosis. Tubulin-binding drugs kill cancerous cells by inhibiting microtubule dynamics, which are required for DNA segregation and therefore cell division.
, there are six members of the tubulin superfamily, although not all are present in all species. Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin (with a mass of ~42 kDa). In contrast, tubulin polymers (microtubules) tend to be much bigger than actin filaments due to their cylindrical nature. Tubulin was long thought to be specific to eukaryotes. More recently, however, several prokaryotic proteins have been shown to be related to tubulin.
<human>: What are some ex

In [43]:
sample = datasets[0]['Background:'].split('<human>:')[0]
print(sample)

 Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins, or one of the member proteins of that superfamily. α- and β-tubulins polymerize into microtubules, a major component of the eukaryotic cytoskeleton. Microtubules function in many essential cellular processes, including mitosis. Tubulin-binding drugs kill cancerous cells by inhibiting microtubule dynamics, which are required for DNA segregation and therefore cell division.
, there are six members of the tubulin superfamily, although not all are present in all species. Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin (with a mass of ~42 kDa). In contrast, tubulin polymers (microtubules) tend to be much bigger than actin filaments due to their cylindrical nature. Tubulin was long thought to be specific to eukaryotes. More recently, however, several prokaryotic proteins have been shown to be related to tubulin.



In [36]:
sample

' Tubulin in molecular biology can refer either to the tubulin protein superfamily of globular proteins, or one of the member proteins of that superfamily. α- and β-tubulins polymerize into microtubules, a major component of the eukaryotic cytoskeleton. Microtubules function in many essential cellular processes, including mitosis. Tubulin-binding drugs kill cancerous cells by inhibiting microtubule dynamics, which are required for DNA segregation and therefore cell division.\n, there are six members of the tubulin superfamily, although not all are present in all species. Both α and β tubulins have a mass of around 50 kDa and are thus in a similar range compared to actin (with a mass of ~42 kDa). In contrast, tubulin polymers (microtubules) tend to be much bigger than actin filaments due to their cylindrical nature. Tubulin was long thought to be specific to eukaryotes. More recently, however, several prokaryotic proteins have been shown to be related to tubulin.\n'

In [44]:
response = gen(sample)
print(response)

<human>:  What is Tubulin in molecular biology? - Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular biology Tubulin in molecular b

#ลอง gen data ของ เจน

In [9]:
import gdown
url = "https://drive.google.com/uc?export=download&id=1ugUZYFBO7UyC11Fce8atVD_sdxXoJ7fE"
output = 'data.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1ugUZYFBO7UyC11Fce8atVD_sdxXoJ7fE
To: /content/data.csv
100%|██████████| 28.9M/28.9M [00:00<00:00, 132MB/s]


'data.csv'

In [10]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,title,description,tags,cover
0,0,Salad Days (Tang LiuZang) - Part 2,The second season of Salad Days (Tang LiuZang).,"['BL', 'Manhua', 'Romance', 'Shounen-ai', 'Spo...",https://cdn.anime-planet.com/manga/primary/sal...
1,1,The Master of Diabolism,As the grandmaster who founded the Demonic Sec...,"['Action', 'Adventure', 'BL', 'Comedy', 'Manhu...",https://cdn.anime-planet.com/manga/primary/the...
2,2,JoJo's Bizarre Adventure Part 7: Steel Ball Run,"Set in 1890, Steel Ball Run spotlights Gyro Ze...","['Action', 'Adventure', 'Horror', 'Mystery', '...",https://cdn.anime-planet.com/manga/primary/joj...
3,3,A Sign of Affection,"Yuki is a typical college student, whose world...","['Romance', 'Shoujo', 'Slice of Life', 'Disabi...",https://cdn.anime-planet.com/manga/primary/a-s...
4,4,Moriarty the Patriot,"Before he was Sherlock’s rival, Moriarty fough...","['Mystery', 'Shounen', 'Detectives', 'England'...",https://cdn.anime-planet.com/manga/primary/mor...


In [11]:
# df[df['title'].str.startswith('z')]

In [12]:
df.iloc[451,2]

'While working as a small supporting actor in a movie, Ki Man Sung has nothing else going for him. However, that was when, through mysterious circumstance, the movie set had become reality during filming. With the sound of the director’s cut, during that short take, he’s not an actor but actually thrown into the events occurring, showing the pain and emotions he faces in that moment. He can only come back if he creates that perfect take…! Can he grow to become the greatest actor?'

In [13]:
gen(df.iloc[451,2])

'<human>:  What happened during the filming of the movie? так Ki Man Sung was a young actor who was working as a small supporting actor in a movie.  \n<bot>:  What happened during the filming of the movie? так Ki Man Sung was a young actor who was working as a small supporting actor in a movie. '

In [14]:
!pip install -q gradio

NotImplementedError: ignored

In [ ]:
"""Nikola Tesla pronounced  10 July  1856 – 7 January 1943
was a Serbian-American inventor,electrical engineer,
mechanical engineer, and futurist best known for 
his contributions to the design of the modern 
alternatingcurrent (AC) electricity supply system."""

In [ ]:
import gradio as gr



demo = gr.Interface(fn=gen, inputs="text", outputs="text")

demo.launch()